Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [240]:
from collections import deque
from random import choice

from tqdm.asyncio import tqdm_asyncio
from tqdm.auto import tqdm
from enum import Enum
import heapq as hq
import numpy as np
import numpy.typing as npt

In [241]:
class Action(Enum):
    LEFT = [[0], [1]]
    RIGHT = [[0], [-1]]
    UP = [[1], [0]]
    DOWN = [[-1], [0]]

class PuzzleState:
    def __init__(self, board: npt.NDArray[np.int_], parent_state = None, action: Action | None = None, depth: int = 0, cost: int = 0, key: int = 0) -> None:
        self.board: npt.NDArray[np.int_] = board
        self.parent_state = parent_state # type: PuzzleState | None
        self.action: Action | None = action
        self.depth: int = depth
        self.cost: int = cost
        self.key: int = key
        self.map: bytes = self.board.tobytes()

    def __lt__(self, other) -> bool:
        # Used for heapq
        return self.key < other.key

class SearchAlgorithm:
    def __init__(self, start_state: PuzzleState) -> None:
        self.start_state: PuzzleState = start_state
        self.goal_state: PuzzleState | None = None
        self.max_frontier_size: int = 0
        self.max_search_depth: int = 0
        self.expanded_nodes: int = 0
        self.path: list[Action] = []

    def reconstruct_path(self) -> None:
        state: PuzzleState = self.goal_state
        while state and state.parent_state:
            self.path.insert(0, state.action)
            state = state.parent_state

In [242]:
PUZZLE_DIM = 3
VALID_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

In [243]:
def is_action_valid(board: npt.NDArray[np.int_], action: Action, action_bounds: tuple[tuple[int, int], tuple[int, int]] = ((-1, -1), (PUZZLE_DIM, PUZZLE_DIM))) -> bool:
    hole_pos = np.nonzero(board == 0)
    target_pos = tuple(np.add(hole_pos, action.value))

    return not (target_pos[0] == action_bounds[0][1] or target_pos[1] == action_bounds[0][0] or target_pos[0] == action_bounds[1][1] or target_pos[1] == action_bounds[1][0])

def available_actions(board: npt.NDArray[np.int_]) -> list[Action]:
    actions: list[Action] = []

    if is_action_valid(board, Action.LEFT):
        actions.append(Action.LEFT)
    if is_action_valid(board, Action.RIGHT):
        actions.append(Action.RIGHT)
    if is_action_valid(board, Action.UP):
        actions.append(Action.UP)
    if is_action_valid(board, Action.DOWN):
        actions.append(Action.DOWN)

    return actions

In [244]:
def get_valid_positions(board: npt.NDArray[np.int_]) -> npt.NDArray[np.bool_]:
    return np.array(board == VALID_STATE)

def get_valid_rows(board: npt.NDArray[np.int_]) -> int:
    if np.argwhere(np.all(get_valid_positions(board), axis=1) == 0).shape == (0, 1):
        return PUZZLE_DIM

    return int(np.cumsum(np.argwhere(np.all(get_valid_positions(board), axis=1) == 0))[0])

def get_valid_columns(board: npt.NDArray[np.int_]) -> int:
    if np.argwhere(np.all(get_valid_positions(board), axis=0) == 0).shape == (0, 1):
        return PUZZLE_DIM

    return int(np.cumsum(np.argwhere(np.all(get_valid_positions(board), axis=0) == 0))[0])

def is_valid(board: npt.NDArray[np.int_]) -> bool:
    return get_valid_rows(board) == PUZZLE_DIM and get_valid_columns(board) == PUZZLE_DIM

In [245]:
def do_action(board: npt.NDArray[np.int_], action: Action, action_bounds: tuple[tuple[int, int], tuple[int, int]] = ((-1, -1), (PUZZLE_DIM, PUZZLE_DIM))) -> np.ndarray:
    if not is_action_valid(board, action, action_bounds):
        return board

    new_state = board.copy()

    hole_pos = np.nonzero(new_state == 0)
    target_pos = tuple(np.add(hole_pos, action.value))

    new_state[hole_pos], new_state[target_pos] = new_state[target_pos], new_state[hole_pos]
    return new_state

In [246]:
# Breadth first search
class BFS(SearchAlgorithm):
    def run(self) -> None:
        visited_boards: set[bytes] = set()
        queue: deque[PuzzleState] = deque([PuzzleState(self.start_state.board)])
        progress: tqdm_asyncio = tqdm(total=1, desc="BFS progress", unit="node", dynamic_ncols=True)

        while queue:
            state: PuzzleState = queue.popleft()
            visited_boards.add(state.map)
            self.expanded_nodes += 1
            state.cost = self.expanded_nodes
            progress.update(1)

            if np.array_equal(state.board, VALID_STATE):
                progress.close()
                self.goal_state = state
                self.reconstruct_path()
                return

            for action in available_actions(state.board):
                new_board: npt.NDArray[np.int_] = do_action(state.board, action)
                new_state: PuzzleState = PuzzleState(new_board, parent_state=state, action=action, depth=state.depth + 1)

                if new_state.map not in visited_boards:
                    queue.append(new_state)
                    visited_boards.add(new_state.map)
                    self.max_frontier_size = max(self.max_frontier_size, len(queue))
                    self.max_search_depth = max(self.max_search_depth, new_state.depth)

        progress.close()

# Depth first search
class DFS(SearchAlgorithm):
    def run(self) -> None:
        visited_boards: set[bytes] = set()
        stack: list[PuzzleState] = [PuzzleState(self.start_state.board)]
        progress: tqdm_asyncio = tqdm(total=1, desc="DFS progress", dynamic_ncols=True)

        while stack:
            state: PuzzleState = stack.pop()
            visited_boards.add(state.map)
            self.expanded_nodes += 1
            state.cost = self.expanded_nodes
            progress.update(1)

            if np.array_equal(state.board, VALID_STATE):
                progress.close()
                self.goal_state = state
                self.reconstruct_path()
                return

            for action in available_actions(state.board):
                new_board: npt.NDArray[np.int_] = do_action(state.board, action)
                new_state: PuzzleState = PuzzleState(new_board, parent_state=state, action=action, depth=state.depth + 1)

                if new_state.map not in visited_boards:
                    stack.append(new_state)
                    visited_boards.add(new_state.map)
                    self.max_frontier_size = max(self.max_frontier_size, len(stack))
                    self.max_search_depth = max(self.max_search_depth, new_state.depth)

        progress.close()

# A*
def manhattan_distance(board: npt.NDArray[np.int_]) -> int:
    distance: int = 0
    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            value = board[i, j]

            if value != 0:
                target_x, target_y = divmod(value - 1, PUZZLE_DIM)
                distance += abs(target_x - i) + abs(target_y - j)

    return distance

class AStar(SearchAlgorithm):
    def run(self) -> None:
        visited_boards: set[bytes] = set()
        initial_key: int = manhattan_distance(self.start_state.board)
        queue: list[PuzzleState] = [PuzzleState(self.start_state.board, key=initial_key)]
        hq.heapify(queue)
        progress: tqdm_asyncio = tqdm(total=1, desc="AStar progress", dynamic_ncols=True)

        while queue:
            state: PuzzleState = hq.heappop(queue)
            self.expanded_nodes += 1
            state.cost = self.expanded_nodes
            progress.update(1)

            if np.array_equal(state.board, VALID_STATE):
                progress.close()
                self.goal_state = state
                self.reconstruct_path()
                return

            for action in available_actions(state.board):
                new_board: npt.NDArray[np.int_] = do_action(state.board, action)
                new_state: PuzzleState = PuzzleState(new_board, parent_state=state, action=action, depth=state.depth + 1)

                if new_state.map not in visited_boards:
                    new_state.key = manhattan_distance(new_state.board)
                    hq.heappush(queue, new_state)
                    visited_boards.add(new_state.map)
                    self.max_frontier_size = max(self.max_frontier_size, len(queue))
                    self.max_search_depth = max(self.max_search_depth, new_state.depth)

        progress.close()

In [247]:
def is_target_at_correct_position(board: npt.NDArray[np.int_], target: int) -> bool:
    return np.all(np.argwhere(board == target) == np.argwhere(VALID_STATE == target))

def place_target_in_position(state: npt.NDArray[np.int_], target: int, position: tuple[int, int], movement_bounds: tuple[tuple[int, int], tuple[int, int]] = ((-1, -1), (PUZZLE_DIM, PUZZLE_DIM)), verbose: bool = False) -> np.ndarray:
    if is_target_at_correct_position(state, target):
        return state

    new_state = state.copy()

    target_pos = np.array(position[::-1])
    if verbose:
        print('target_pos', target_pos)
        print('aligning gap')

    # Place gap at the edge of the region defined by the target current position and its correct position.
    while True:
        gap_position = tuple(np.argwhere(new_state == 0).flatten())[::-1]
        current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
        if (
            gap_position[0] == current_position[0] or gap_position[1] == current_position[1] or
            gap_position[0] == position[0] or gap_position[1] == position[1] or
            is_target_at_correct_position(new_state, target)
        ):
            break

        hole_pos = np.argwhere(new_state == 0).flatten()
        starting_pos = np.argwhere(new_state == target).flatten()

        if verbose:
            print(new_state)
            print('hole_pos', hole_pos)
            print('starting_pos', starting_pos)

        positions = np.array([hole_pos, starting_pos]).reshape(2, 2)

        min_values = np.min(positions, axis=0)
        max_values = np.max(positions, axis=0)

        min_y, min_x = tuple(min_values)
        max_y, max_x = tuple(max_values)

        min_x -= 1
        min_y -= 1
        max_x += 1
        max_y += 1

        if verbose:
            print('top_left', (min_x, min_y))
            print('bottom_right', (max_x, max_y))

        if min_x < movement_bounds[0][0]:
            min_x = movement_bounds[0][0]
        if min_y < movement_bounds[0][1]:
            min_y = movement_bounds[0][1]
        if max_x > movement_bounds[1][0]:
            max_x = movement_bounds[1][0]
        if max_y > movement_bounds[1][1]:
            max_y = movement_bounds[1][1]

        if max_x - min_x == 2:
            if max_x == PUZZLE_DIM:
                min_x -= 1
            else:
                max_x += 1
        if max_y - min_y == 2:
            if max_y == PUZZLE_DIM:
                min_y -= 1
            else:
                max_y += 1

        bounds = ((min_x, min_y), (max_x, max_y))
        if verbose:
            print('movement_bounds', movement_bounds)
            print('bounds', bounds)

        if not is_action_valid(new_state, Action.DOWN, bounds):
            new_state = do_action(new_state, Action.RIGHT, bounds)
            if verbose:
                print(new_state)

            gap_position = tuple(np.argwhere(new_state == 0).flatten())[::-1]
            current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
            if (
                gap_position[0] == current_position[0] or gap_position[1] == current_position[1] or
                gap_position[0] == position[0] or gap_position[1] == position[1] or
                is_target_at_correct_position(new_state, target)
            ):
                break

        if not is_action_valid(new_state, Action.LEFT, bounds):
            new_state = do_action(new_state, Action.DOWN, bounds)
            if verbose:
                print(new_state)

            gap_position = tuple(np.argwhere(new_state == 0).flatten())[::-1]
            current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
            if (
                gap_position[0] == current_position[0] or gap_position[1] == current_position[1] or
                gap_position[0] == position[0] or gap_position[1] == position[1] or
                is_target_at_correct_position(new_state, target)
            ):
                break

        if not is_action_valid(new_state, Action.UP, bounds):
            new_state = do_action(new_state, Action.LEFT, bounds)
            if verbose:
                print(new_state)

            gap_position = tuple(np.argwhere(new_state == 0).flatten())[::-1]
            current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
            if (
                gap_position[0] == current_position[0] or gap_position[1] == current_position[1] or
                gap_position[0] == position[0] or gap_position[1] == position[1] or
                is_target_at_correct_position(new_state, target)
            ):
                break

        if not is_action_valid(new_state, Action.RIGHT, bounds):
            new_state = do_action(new_state, Action.UP, bounds)
            if verbose:
                print(new_state)

            gap_position = tuple(np.argwhere(new_state == 0).flatten())[::-1]
            current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
            if (
                gap_position[0] == current_position[0] or gap_position[1] == current_position[1] or
                gap_position[0] == position[0] or gap_position[1] == position[1] or
                is_target_at_correct_position(new_state, target)
            ):
                break
    if verbose:
        print('gap aligned')
        print('aligning_target')

    # Place target in position
    while True:
        hole_pos = np.argwhere(new_state == 0).flatten()
        starting_pos = np.argwhere(new_state == target).flatten()

        if verbose:
            print(new_state)
            print('hole_pos', hole_pos)
            print('starting_pos', starting_pos)
            print('target_pos', target_pos)

        positions = np.array([hole_pos, starting_pos, target_pos]).reshape(3, 2)

        min_values = np.min(positions, axis=0)
        max_values = np.max(positions, axis=0)

        min_y, min_x = tuple(min_values)
        max_y, max_x = tuple(max_values)

        min_x -= 1
        min_y -= 1
        max_x += 1
        max_y += 1

        if verbose:
            print('top_left', (min_x, min_y))
            print('bottom_right', (max_x, max_y))

        if min_x < movement_bounds[0][0]:
            min_x = movement_bounds[0][0]
        if min_y < movement_bounds[0][1]:
            min_y = movement_bounds[0][1]
        if max_x > movement_bounds[1][0]:
            max_x = movement_bounds[1][0]
        if max_y > movement_bounds[1][1]:
            max_y = movement_bounds[1][1]

        if max_x - min_x == 2:
            if max_x == PUZZLE_DIM:
                min_x -= 1
            else:
                max_x += 1
        if max_y - min_y == 2:
            if max_y == PUZZLE_DIM:
                min_y -= 1
            else:
                max_y += 1

        bounds = ((min_x, min_y), (max_x, max_y))
        if verbose:
            print('movement_bounds', movement_bounds)
            print('bounds', bounds)

        if not is_action_valid(new_state, Action.DOWN, bounds):
            new_state = do_action(new_state, Action.RIGHT, bounds)
            if verbose:
                print(new_state)

            current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
            if current_position == position or is_target_at_correct_position(new_state, target):
                break

        if not is_action_valid(new_state, Action.LEFT, bounds):
            new_state = do_action(new_state, Action.DOWN, bounds)
            if verbose:
                print(new_state)

            current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
            if current_position == position or is_target_at_correct_position(new_state, target):
                break

        if not is_action_valid(new_state, Action.UP, bounds):
            new_state = do_action(new_state, Action.LEFT, bounds)
            if verbose:
                print(new_state)

            current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
            if current_position == position or is_target_at_correct_position(new_state, target):
                break

        if not is_action_valid(new_state, Action.RIGHT, bounds):
            new_state = do_action(new_state, Action.UP, bounds)
            if verbose:
                print(new_state)

            current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
            if current_position == position or is_target_at_correct_position(new_state, target):
                break
    if verbose:
        print('target_aligned')

    return new_state

def place_target_in_correct_position(state: npt.NDArray[np.int_], target: int, verbose=False) -> np.ndarray:
    target_y, target_x = tuple(np.argwhere(VALID_STATE == target).flatten())
    if is_target_at_correct_position(state, target):
        return state

    new_state = state.copy()

    building_row: bool = (get_valid_rows(new_state) == target_y and target_x != 0) or (target_x, target_y) == (0, 0)

    solving_region: tuple[tuple[int, int], tuple[int, int]] = ((target_x - 1, target_y - 1), (PUZZLE_DIM, PUZZLE_DIM))
    pivoting_region: tuple[tuple[int, int], tuple[int, int]] = ((get_valid_columns(new_state) - 1 + int(not building_row), get_valid_rows(new_state) - 1 + int(building_row and target_x != 0)), (PUZZLE_DIM, PUZZLE_DIM))

    if verbose:
        print('target', target)
        print('solving_region', solving_region)
        print('pivoting_region', pivoting_region)
        print('building_row', building_row)

    if target_x == PUZZLE_DIM - 1 or target_y == PUZZLE_DIM - 1:
        # Corner case

        # Position gap inside solving region
        new_state = do_action(new_state, Action.UP)
        if is_target_at_correct_position(new_state, target):
            return new_state

        new_state = do_action(new_state, Action.LEFT)
        if is_target_at_correct_position(new_state, target):
            return new_state

        offset = (-1, 1) if target_x == PUZZLE_DIM - 1 else (1, -1)
        pivot = tuple(np.add((target_x, target_y), offset))

        if verbose:
            print('pivot', pivot)

        # Placing target at pivot
        new_state = place_target_in_position(new_state, target, pivot, pivoting_region, verbose)

        # Placing gap at row below/column to the right of the pivot (building row/column)
        while True:
            gap_y, gap_x = tuple(np.argwhere(new_state == 0).flatten())

            if verbose:
                print(new_state)

            if building_row:
                if gap_y == pivot[1] + 1:
                    break

                if verbose:
                    print('aligning gap to target')

                if gap_y < pivot[1] + 1:
                    new_state = do_action(new_state, Action.UP)
                else:
                    new_state = do_action(new_state, Action.DOWN)
            else:
                if gap_x == pivot[0] + 1:
                    break

                if verbose:
                    print('aligning gap to target')

                if gap_x < pivot[0] + 1:
                    new_state = do_action(new_state, Action.LEFT)
                else:
                    new_state = do_action(new_state, Action.RIGHT)

        # Placing gap at cell to the left/above the pivot (building row/column)
        while True:
            gap_y, gap_x = tuple(np.argwhere(new_state == 0).flatten())

            if verbose:
                print('new_state')
                print(new_state)

            if building_row and gap_x == pivot[0] - 1 or not building_row and gap_y == pivot[1] - 1:
                break

            if verbose:
                print('adjusting gap')

            if building_row:
                new_state = do_action(new_state, Action.RIGHT)
            else:
                new_state = do_action(new_state, Action.DOWN)

        # Gap final placement
        if building_row:
            new_state = do_action(new_state, Action.DOWN)
        else:
            new_state = do_action(new_state, Action.RIGHT)

        # Fixed movements
        if building_row:
            new_state = do_action(new_state, Action.DOWN)
            new_state = do_action(new_state, Action.LEFT)
            new_state = do_action(new_state, Action.UP)
            new_state = do_action(new_state, Action.LEFT)
            new_state = do_action(new_state, Action.DOWN)
            new_state = do_action(new_state, Action.RIGHT)
            new_state = do_action(new_state, Action.RIGHT)
            new_state = do_action(new_state, Action.UP)
        else:
            new_state = do_action(new_state, Action.RIGHT)
            new_state = do_action(new_state, Action.UP)
            new_state = do_action(new_state, Action.LEFT)
            new_state = do_action(new_state, Action.UP)
            new_state = do_action(new_state, Action.RIGHT)
            new_state = do_action(new_state, Action.DOWN)
            new_state = do_action(new_state, Action.DOWN)
            new_state = do_action(new_state, Action.LEFT)

        if pivot == (PUZZLE_DIM - 2, PUZZLE_DIM - 2) and not building_row:
            new_state = do_action(new_state, Action.LEFT)
            new_state = do_action(new_state, Action.UP)

        return new_state

    current_y, current_x = tuple(np.argwhere(new_state == target).flatten())
    if current_x <= solving_region[0][0] or current_y <= solving_region[0][1]:
        # Position gap inside pivoting region
        while True:
            gap_y, gap_x = tuple(np.argwhere(new_state == 0).flatten())

            if verbose:
                print(new_state)

            if gap_x > pivoting_region[0][0] and gap_y > pivoting_region[0][1]:
                if verbose:
                    print('gap inside region')

                break

            if verbose:
                print('adjusting gap')
            new_state = do_action(new_state, Action.UP)
            if is_target_at_correct_position(new_state, target):
                return new_state

            new_state = do_action(new_state, Action.LEFT)
            if is_target_at_correct_position(new_state, target):
                return new_state

        if verbose:
            print('adjusting target')

        offset = (int(not building_row), int(building_row))
        target_pos = tuple(np.add((target_x, target_y), offset))

        if verbose:
            print('target_pos', target_pos)

        new_state = place_target_in_position(new_state, target, target_pos, pivoting_region, verbose)
        if is_target_at_correct_position(new_state, target):
            return new_state

    if verbose:
        print('target inside region')

    # Position gap inside solving region
    while True:
        gap_y, gap_x = tuple(np.argwhere(new_state == 0).flatten())

        if verbose:
            print('new_state')
            print(new_state)

        if gap_x > solving_region[0][0] and gap_y > solving_region[0][1]:
            if verbose:
                print('gap inside region')

            break

        if verbose:
            print('adjusting gap')

        new_state = do_action(new_state, Action.UP)
        if is_target_at_correct_position(new_state, target):
            return new_state

        new_state = do_action(new_state, Action.LEFT)
        if is_target_at_correct_position(new_state, target):
            return new_state

    current_y, current_x = tuple(np.argwhere(new_state == target).flatten())
    if current_x <= solving_region[0][0] or current_y <= solving_region[0][1]:
        # Position gap inside pivoting region
        while True:
            gap_y, gap_x = tuple(np.argwhere(new_state == 0).flatten())

            if verbose:
                print(new_state)

            if gap_x > pivoting_region[0][0] and gap_y > pivoting_region[0][1]:
                if verbose:
                    print('gap inside region')

                break

            if verbose:
                print('adjusting gap')
            new_state = do_action(new_state, Action.UP)
            if is_target_at_correct_position(new_state, target):
                return new_state

            new_state = do_action(new_state, Action.LEFT)
            if is_target_at_correct_position(new_state, target):
                return new_state

        if verbose:
            print('adjusting target')

        offset = (int(not building_row), int(building_row))
        target_pos = tuple(np.add((target_x, target_y), offset))

        if verbose:
            print('target_pos', target_pos)

        new_state = place_target_in_position(new_state, target, target_pos, pivoting_region, verbose)
        if is_target_at_correct_position(new_state, target):
            return new_state

    if verbose:
        print('target inside region')

    # Position gap inside solving region
    while True:
        gap_y, gap_x = tuple(np.argwhere(new_state == 0).flatten())

        if verbose:
            print('new_state')
            print(new_state)

        if gap_x > solving_region[0][0] and gap_y > solving_region[0][1]:
            if verbose:
                print('gap inside region')

            break

        if verbose:
            print('adjusting gap')

        new_state = do_action(new_state, Action.UP)
        if is_target_at_correct_position(new_state, target):
            return new_state

        new_state = do_action(new_state, Action.LEFT)
        if is_target_at_correct_position(new_state, target):
            return new_state

    new_state = place_target_in_position(new_state, target, (target_x, target_y), solving_region, verbose)

    return new_state

In [248]:
def solve(state: npt.NDArray[np.int_]):
    new_state = state.copy()

    for i in range(PUZZLE_DIM):
        new_state = place_target_in_correct_position(new_state, i + 1)

    for k in range(1, PUZZLE_DIM - 1):
        for i in range(PUZZLE_DIM - k):
            new_state = place_target_in_correct_position(new_state, k + PUZZLE_DIM * (i + k))
        for i in range(PUZZLE_DIM - k):
            new_state = place_target_in_correct_position(new_state, k + i + 1 + PUZZLE_DIM * k)

    new_state = place_target_in_correct_position(new_state, PUZZLE_DIM * PUZZLE_DIM - 1)

    return new_state

In [249]:
RANDOMIZE_STEPS = 100000

initial_state = VALID_STATE
for _ in tqdm(range(RANDOMIZE_STEPS)):
    initial_state = do_action(initial_state, choice(available_actions(initial_state)))

  0%|          | 0/100000 [00:00<?, ?it/s]

In [250]:
initial_state

array([[6, 5, 2],
       [3, 0, 8],
       [7, 1, 4]])

In [251]:
BFS(PuzzleState(initial_state)).run()

BFS progress:   0%|          | 0/1 [00:00<?, ?node/s]

In [252]:
DFS(PuzzleState(initial_state)).run()

DFS progress:   0%|          | 0/1 [00:00<?, ?it/s]

In [253]:
AStar(PuzzleState(initial_state)).run()

AStar progress:   0%|          | 0/1 [00:00<?, ?it/s]

In [254]:
solve(initial_state)

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 0]])